In [2]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection, metrics, datasets
from neupy import algorithms, layers, environment
import pickle
environment.reproducible()
environment.speedup()


def load_data():
    mnist = datasets.fetch_mldata('MNIST original')
    data = mnist.data

    target_scaler = OneHotEncoder()
    target = mnist.target.reshape((-1, 1))
    target = target_scaler.fit_transform(target).todense()

    n_samples = data.shape[0]
    data = data.reshape((n_samples, 1, 28, 28))

    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        data.astype(np.float32),
        target.astype(np.float32),
        train_size=(6 / 7.)
    )

    mean = x_train.mean(axis=(0, 2, 3))
    std = x_train.std(axis=(0, 2, 3))

    #x_train -= mean
    #x_train /= std
    #x_test -= mean
    #x_test /= std
    x_train /= 255.0
    x_test /= 255.0

    return x_train, x_test, y_train, y_test

In [3]:
x_train, x_test, y_train, y_test = load_data()

/Users/user/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
network = algorithms.Adadelta(
    [
        layers.Input((1, 28, 28)),

        layers.Convolution((32, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.Convolution((48, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.MaxPooling((2, 2)),

        layers.Convolution((64, 3, 3)) > layers.BatchNorm() > layers.Relu(),
        layers.MaxPooling((2, 2)),

        layers.Reshape(),
        layers.Linear(1024) > layers.BatchNorm() > layers.Relu(),
        layers.Softmax(10),
    ],

    # Using categorical cross-entropy as a loss function
    error='categorical_crossentropy',

    # Min-batch size
    batch_size=128,

    # Learning rate. We can allow high values
    # since we are using Batch Normalization
    step=1.0,

    # Shows information about algorithm and
    # training progress in terminal
    verbose=True,

    # Randomly shuffles training dataset before every epoch
    shuffle_data=True,

    # Step decay algorithm minimizes learning step
    # monotonically after each iteration.
    addons=[algorithms.StepDecay],
    # Parameter controls step redution frequency. The higher
    # the value the slower step parameter decreases.
    reduction_freq=8,
)

# Shows networks architecture in terminal's output
network.architecture()

# Train for only two epochs
network.train(x_train, y_train, x_test, y_test, epochs=1)

y_predicted = network.predict(x_test).argmax(axis=1)
y_test_labels = np.asarray(y_test.argmax(axis=1)).reshape(len(y_test))

print(metrics.classification_report(y_test_labels, y_predicted))
score = metrics.accuracy_score(y_test_labels, y_predicted)
print("Validation accuracy: {:.2%}".format(score))


Main information

[ALGORITHM] Adadelta

[OPTION] batch_size = 128
[OPTION] verbose = True
[OPTION] epoch_end_signal = None
[OPTION] show_epoch = 1
[OPTION] shuffle_data = True
[OPTION] step = 1.0
[OPTION] train_end_signal = None
[OPTION] error = categorical_crossentropy
[OPTION] addons = ['StepDecay']
[OPTION] decay = 0.95
[OPTION] epsilon = 1e-05
[OPTION] reduction_freq = 8

[THEANO] Initializing Theano variables and functions.
[THEANO] Initialization finished successfully. It took 18.88 seconds

Network's architecture

--------------------------------------------------
| #  | Input shape  | Layer type  | Output shape |
--------------------------------------------------
|  1 |  (1, 28, 28) |       Input |  (1, 28, 28) |
|  2 |  (1, 28, 28) | Convolution | (32, 26, 26) |
|  3 | (32, 26, 26) |   BatchNorm | (32, 26, 26) |
|  4 | (32, 26, 26) |        Relu | (32, 26, 26) |
|  5 | (32, 26, 26) | Convolution | (48, 24, 24) |
|  6 | (48, 24, 24) |   BatchNorm | (48, 24, 24) |
|  7 | (48, 2

Time: 0:00:00 |N/A%|                           | ETA:  --:--:-- | error: ------



Start training

[TRAINING DATA] shapes: (60000, 1, 28, 28)
[TEST DATA] shapes: (10000, 1, 28, 28)
[TRAINING] Total epochs: 1

---------------------------------------------------------
|    Epoch    |  Train err  |  Valid err  |    Time     |
---------------------------------------------------------


Time: 0:00:00 |N/A%|                                           | ETA:  --:--:--

|           1 |    0.099958 |    0.063426 |       06:55 |
---------------------------------------------------------



             precision    recall  f1-score   support

          0       0.97      1.00      0.98       936
          1       0.98      1.00      0.99      1163
          2       0.98      0.99      0.98       982
          3       1.00      0.99      0.99      1038
          4       0.98      0.99      0.99       948
          5       1.00      0.95      0.97       921
          6       0.97      0.99      0.98      1013
          7       0.99      0.98      0.98      1029
          8       0.99      0.96      0.97       978
          9       0.98      0.98      0.98       992

avg / total       0.98      0.98      0.98     10000

Validation accuracy: 98.20%


In [6]:
import pickle
f = open("net.dump", "wb")
pickle.dump(network, f)
f.close()

In [1]:
import pickle
f = open("net.dump", "r")
network = pickle.load(f)
f.close()


Main information

[ALGORITHM] Adadelta

[OPTION] batch_size = 128
[OPTION] verbose = True
[OPTION] epoch_end_signal = None
[OPTION] show_epoch = 1
[OPTION] shuffle_data = True
[OPTION] step = 1.0
[OPTION] train_end_signal = None
[OPTION] error = categorical_crossentropy
[OPTION] addons = ['StepDecay']
[OPTION] decay = 0.95
[OPTION] epsilon = 1e-05
[OPTION] reduction_freq = 8

[THEANO] Initializing Theano variables and functions.
[THEANO] Initialization finished successfully. It took 20.48 seconds


In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(15, 0.1)

while(True):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    cv2.imshow('frame', rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #newframe = cv2.resize(frame,(28,28))
        #out = cv2.imwrite('capture.jpg', frame)
        #newout = cv2.imwrite('newcapture.jpg', newframe)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray,(28,28))
        ret, gb = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
        gb = cv2.bitwise_not(gb)        
        break

cap.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)

gb = gb/255
M = network.predict(np.array([[gb]]))[0]
args = M.argsort()[-2:]
print "Скорее всего это цифра ", args[1]
print "Но может быть и ", args[0]

Скорее всего это цифра  4
Но может быть и  1


In [3]:
from skimage import io
from skimage import img_as_float
from skimage.color import rgb2gray
from skimage.transform import resize
def img_ready(filename):
    im = io.imread(filename)
    imc = rgb2gray(1-im)
    imcb = imc>0.5
    imr=resize(imcb, (28, 28), mode='reflect')
    return imr

In [1]:
network.predict(np.array([[img]]))

NameError: name 'network' is not defined

In [3]:
M

array([  3.04910960e-03,   9.89098012e-01,   1.26404129e-05,
         3.49185086e-07,   7.48948660e-03,   3.08556923e-06,
         1.86622041e-04,   1.05273859e-04,   5.19283967e-05,
         3.50924574e-06], dtype=float32)